In [2]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 4.3 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=3b171cffdb7eefc3089a7c43b31055ce03ebf783894bf02c19247ce2da902010
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.5.0


In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [8]:
train_images = train_images/255.0
test_images = test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [11]:
def build_model(hp):     #hp is hyperparameter
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value=64, max_value = 128, step = 16),
        kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
        activation = 'relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value=32, max_value = 64, step = 16),
        kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
        activation = 'relu'
    ), 
    keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')                      
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model



In [12]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', 
                          max_trials=5, 
                          directory='output', 
                          project_name='mnist_fashion')

In [14]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.1054999977350235

Best val_accuracy So Far: 0.8368333578109741
Total elapsed time: 00h 02m 53s
INFO:tensorflow:Oracle triggered exit


In [16]:
#get best model from all trials
model =tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3539072   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,589,914
Trainable params: 3,589,914
Non-trainable params: 0
_________________________________________________________________


In [19]:
#now retrain all images again consedring this particular model
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3039 - accuracy: 0.8882 - val_loss: 0.3296 - val_accuracy: 0.8782
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2941 - accuracy: 0.8909 - val_loss: 0.3454 - val_accuracy: 0.8737
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2865 - accuracy: 0.8931 - val_loss: 0.3384 - val_accuracy: 0.8808
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2808 - accuracy: 0.8954 - val_loss: 0.3350 - val_accuracy: 0.8790
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2735 - accuracy: 0.8973 - val_loss: 0.3333 - val_accuracy: 0.8778
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2665 - accuracy: 0.8991 - val_loss: 0.3216 - val_accuracy: 0.8840
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2596 - accuracy: 0.9037 - val_loss: 0.3123 - val_accuracy